In [1]:
from dotenv import load_dotenv
load_dotenv("openai.env")

#使用openai的官方sdk
import openai
import os

openai.api_base =os.environ.get("OPENAI_PROXY")
openai.api_key = os.environ.get("OPENAI_API_KEY")

messages = [
{"role": "user", "content": "介绍下你自己"}
]

res = openai.ChatCompletion.create(
model="gpt-4-1106-preview",
messages=messages,
stream=False,
)

print(res['choices'][0]['message']['content'])

您好，我是一个人工智能助手，由 OpenAI 创建。我的设计是基于最新的机器学习技术，并通过 GPT-3 模型（Generative Pretrained Transformer 3）实现能够理解和生成自然语言文本的能力。我可以在多种任务中协助您，比如回答问题、提供信息、帮助解决问题、学习新知识、进行文本创作等。

我的知识截止于 2023 年左右，这意味着我在那个时间点之前的历史、事实信息和公认的知识都很熟悉。但是，请注意，我并非实时更新的，并且我没有个人经验或意识，因此我提供的信息是基于数据库和算法生成的，而不是任何个人见解或感知。我的设计宗旨是提供有帮助、准确和中立的回答。如果您有任何问题或需要协助，我随时在这里为您服务！


# Lost in the middle: 长上下文精度问题
<hr>

In [2]:
! pip install sentence-transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain.chains import LLMChain,StuffDocumentsChain
from langchain.document_transformers import (
    LongContextReorder
)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import  Chroma

#使用huggingface托管的开源LLM来做嵌入，MiniLM-L6-v2是一个较小的LLM 
embedings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登之环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里.伯德是一位标志性的NBA球员。"
]

retrieval = Chroma.from_texts(text,embedings).as_retriever(
    search_kwargs={"k": 10}
)
query = "关于我的喜好都知道什么?"

#根据相关性返回文本块
docs = retrieval.get_relevant_documents(query)
docs


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content='凯尔特人队是我最喜欢的球队。', metadata={}),
 Document(page_content='我非常喜欢去看电影。', metadata={}),
 Document(page_content='带我飞往月球是我最喜欢的歌曲之一。', metadata={}),
 Document(page_content='篮球是一项伟大的运动。', metadata={}),
 Document(page_content='拉里.伯德是一位标志性的NBA球员。', metadata={}),
 Document(page_content='这只是一段随机的文字。', metadata={}),
 Document(page_content='这是一篇关于波士顿凯尔特人的文件。', metadata={}),
 Document(page_content='《艾尔登之环》是过去15年最好的游戏之一。', metadata={}),
 Document(page_content='波士顿凯尔特人队以20分的优势赢得了比赛。', metadata={}),
 Document(page_content='L.科内特是凯尔特人队最好的球员之一。', metadata={})]

In [4]:
#对检索结果进行重新排序，根据论文的方案
#问题相关性越低的内容块放在中间
#问题相关性越高的内容块放在头尾

reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs)

#头尾共有4个高相关性内容块
reo_docs

[Document(page_content='我非常喜欢去看电影。', metadata={}),
 Document(page_content='篮球是一项伟大的运动。', metadata={}),
 Document(page_content='这只是一段随机的文字。', metadata={}),
 Document(page_content='《艾尔登之环》是过去15年最好的游戏之一。', metadata={}),
 Document(page_content='L.科内特是凯尔特人队最好的球员之一。', metadata={}),
 Document(page_content='波士顿凯尔特人队以20分的优势赢得了比赛。', metadata={}),
 Document(page_content='这是一篇关于波士顿凯尔特人的文件。', metadata={}),
 Document(page_content='拉里.伯德是一位标志性的NBA球员。', metadata={}),
 Document(page_content='带我飞往月球是我最喜欢的歌曲之一。', metadata={}),
 Document(page_content='凯尔特人队是我最喜欢的球队。', metadata={})]

In [5]:
from dotenv import load_dotenv
load_dotenv("openai.env")
import os
api_key = os.environ.get("OPENAI_API_KEY")
api_base = os.environ.get("OPENAI_API_BASE")

In [6]:
#检测下这种方案的精度效果
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

#设置llm
llm = OpenAI(
    api_key=api_key,
    api_base=api_base,
    model="gpt-3.5-turbo-instruct",
    temperature=0
)

document_prompt = PromptTemplate(
    input_variables=["page_content"],template="{page_content}"
)

stuff_prompt_override ="""Given this text extracts:
----------------------------------------
{context}
----------------------------------------
Please answer the following questions:
{query}
"""

prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context","query"]
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

WorkChain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)

#调用
WorkChain.run(
    input_documents=reo_docs,
    query="我最喜欢做什么事情？"
)


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain/utils/utils.py:155: UserWarning: WARNING! api_key is not default parameter.
                api_key was transferred to model_kwargs.
                Please confirm that api_key is what you intended.
  warnings.warn(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain/utils/utils.py:155: UserWarning: WARNING! api_base is not default parameter.
                api_base was transferred to model_kwargs.
                Please confirm that api_base is what you intended.
  warnings.warn(


'\n看电影。'